<a href="https://colab.research.google.com/github/Sthesha/Sthesh/blob/master/Project%20Hybrid%20Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import gensim
from gensim.models import Word2Vec
from nltk.corpus import stopwords 
import nltk
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
main_path = "/root/nltk_data/corpora/wordnet.zip" # Specify the path
import os
if os.path.exists(main_path)== False:
  ! unzip /root/nltk_data/corpora/wordnet.zip -d /root/nltk_data/
os.listdir('/root/nltk_data')
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline, make_pipeline
# Below libraries are for feature representation using sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Below libraries are for similarity matrices using sklearn
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
import copy
from IPython.display import clear_output
import warnings
from re import sub
import seaborn as sns
from wordcloud import WordCloud
from sklearn import metrics
from sklearn.model_selection import train_test_split
from PIL import Image 
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from xgboost import XGBClassifier
import random
import nltk
from gensim.models import KeyedVectors
nltk.download('omw-1.4')
warnings.filterwarnings("ignore")

import re
from google.colab import auth
import os

import numpy as np
from sklearn.metrics import ndcg_score, average_precision_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
os.listdir('/root/nltk_data')

['tokenizers', 'corpora', 'wordnet']

In [ ]:

# !curl https://sdk.cloud.google.com | bash
# !gcloud init

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
main_path = "/content/mind-news-dataset.zip" # Specify the path

# Check whether the specified path exists or not
if os.path.exists(main_path)== False:
  ! kaggle datasets download arashnic/mind-news-dataset
  ! unzip /content/mind-news-dataset.zip


In [ ]:
news_data = pd.read_csv('/content/news.tsv/news.tsv', header=None, sep="\t")
news_data.columns = ['News ID', "Category", "SubCategory", "Title", "Abstract", "URL", "Title Entities" , "Abstract Entities"]
news_data = news_data.drop(['URL', 'Title Entities', 'SubCategory',  'Abstract Entities'], axis=1)
news_data

,News ID,Category,Title,Abstract
0,N55528,lifestyle,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the..."
1,N19639,health,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...
2,N61837,news,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...
3,N53526,health,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi..."
4,N38324,health,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re..."
...,...,...,...,...
51277,N16909,weather,"Adapting, Learning And Soul Searching: Reflect...",Woolsey Fire Anniversary: A community is forev...
51278,N47585,lifestyle,Family says 13-year-old Broadway star died fro...,NaN
51279,N7482,sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b..."
51280,N34418,sports,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ..."


In [ ]:
print('the number of articles before processing :',len(news_data))
news_data.drop_duplicates(subset=["Abstract"],inplace=True)
print('The number of articles after processing :',len(news_data))
news_data.info()

the number of articles before processing : 51282
The number of articles after processing : 47310
<class 'pandas.core.frame.DataFrame'>
Int64Index: 47310 entries, 0 to 51280
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   News ID   47310 non-null  object
 1   Category  47310 non-null  object
 2   Title     47310 non-null  object
 3   Abstract  47309 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [ ]:
#upload the reddit comments dataset
userComments = pd.read_csv('/content/reddit_data.csv', sep=",")
userComments.head()

,parent_id,text,topic,length,size_range,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,t1_crojgfu,Thanks! Not sure if those links were up there ...,pcmasterrace,103,101 to 200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,t1_cquq97y,I think its unlikely someone would kill them s...,news,163,101 to 200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,t1_cr92xnl,Hoult is another one that's important.\n\nBut ...,movies,99,0 to 100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,t1_crk6sin,Can have my opinion.\n\nThey're noisy and hot....,pcmasterrace,1556,501 +,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,t1_cr9398p,Nice! That reminds me of a more recent article...,news,252,201 to 500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#remove the unneccessary columns
userComments = userComments.drop(userComments.loc[:, 'length':'Unnamed: 32'].columns, axis=1)
userComments

,parent_id,text,topic
0,t1_crojgfu,Thanks! Not sure if those links were up there ...,pcmasterrace
1,t1_cquq97y,I think its unlikely someone would kill them s...,news
2,t1_cr92xnl,Hoult is another one that's important.\n\nBut ...,movies
3,t1_crk6sin,Can have my opinion.\n\nThey're noisy and hot....,pcmasterrace
4,t1_cr9398p,Nice! That reminds me of a more recent article...,news
...,...,...,...
44219,t1_cr4jdga,But aren't most of the illegal immigrants Cath...,news
44220,t1_cr98ybu,Yes because she held him down and forced the c...,news
44221,t1_crbzx6n,Six String Samurai is one of my favorite movie...,movies
44222,t1_crozy8f,nah this is the kind of comments you get when ...,nfl


In [ ]:
#remove rows with null values
print('the number of user comments before processing :',len(userComments))
userComments = userComments.dropna().reset_index(drop=True)
print('The number of user comments after processing :', len(userComments))

the number of user comments before processing : 44224
The number of user comments after processing : 39413


In [ ]:
#remove the stopwords , words such as i.e {this, or, the}
def remove_stopwords(sentence):
    stop_words = set(stopwords.words('english'))
    processedSentence = []
    word_tokens = word_tokenize(sentence)
    for i in word_tokens:
        if i not in stop_words:
            processedSentence.append(i)
    return processedSentence

In [ ]:
lemmatizer = WordNetLemmatizer() 
alphaPattern = "[^a-zA-Z0-9<>]"
numerical = comment = "[0-9]+"

# Making a function to lemmatize all the words
lemmatizer = WordNetLemmatizer() 

def lemmatize_column(data, col_name):
        processedSentence = []
        sentences = data[col_name]
        for sentence in sentences:
            sentence = sentence.lower()
            sentence = re.sub(alphaPattern, " ", sentence)
            sentence = re.sub(numerical, " ", sentence)

            #call the stopword method
            words = remove_stopwords(sentence)
            holder = []
            for word in words:
                w = word
                word =lemmatizer.lemmatize(word)
                if(len(word)>2):
                  holder.append(word)
            processedSentence.append(' '.join(holder))
        data[col_name]=processedSentence
        return data

In [ ]:
news_data.isna().sum()
news_data.dropna(inplace=True)

In [ ]:
#apply the lemmatisation method and remove.
data_before = lemmatize_column(news_data, "Abstract")

In [ ]:
data_before

,News ID,Category,Title,Abstract
0,N55528,lifestyle,"The Brands Queen Elizabeth, Prince Charles, an...",shop notebook jacket royal live without
1,N19639,health,50 Worst Habits For Belly Fat,seemingly harmless habit holding back keeping ...
2,N61837,news,The Cost of Trump's Aid Freeze in the Trenches...,ivan molchanets peeked parapet sand bag front ...
3,N53526,health,I Was An NBA Wife. Here's How It Affected My M...,felt like fraud nba wife help fact nearly dest...
4,N38324,health,"How to Get Rid of Skin Tags, According to a De...",seem harmless good reason ignore post get rid ...
...,...,...,...,...
51274,N43432,news,US Forest Service shuts down vandalized Georgi...,gainesville forest service say vandalism forci...
51275,N17258,news,Realme takes chunk of India mobile market as S...,percent phone shipped year year
51276,N23858,sports,Young Northeast Florida fans flock to U.S. wom...,woman national soccer team arrived jacksonvill...
51279,N7482,sports,St. Dominic soccer player tries to kick cancer...,sometimes happens sideline even important happ...


In [ ]:
# #limit while working on to 1000
# news_data = news_data.truncate(before=0, after=10000)

In [ ]:
#upload the user behaviors dataset
users_behavior = pd.read_csv('/content/MINDsmall_train/behaviors.tsv', header=None, sep="\t")
users_behavior.columns = ["Impression", "User ID", "Time", "History", "Impressions"]
users_behavior.head()

,Impression,User ID,Time,History,Impressions
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...


In [ ]:
#drop the unneccesary columns
users_behavior = users_behavior.drop(['Time', 'Impression'], axis=1)


In [ ]:
# df.groupby('User ID', as_index=False).agg({'User ID' : 'first', 'History' : ' '.join})

In [ ]:
print('the number of articles before processing :',len(users_behavior))
users_behavior=users_behavior.dropna().reset_index(drop=True)
print('The number of articles after processing :', len(users_behavior))


the number of articles before processing : 156965
The number of articles after processing : 153727


In [ ]:
user_impressions = users_behavior

In [ ]:
users_behavior.head(10)

,User ID,History,Impressions
0,U13740,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,U91836,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,U73700,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,U34670,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
4,U8125,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...
5,U19739,N39074 N14343 N32607 N32320 N22007 N442 N19001...,N21119-1 N53696-0 N33619-1 N25722-0 N2869-0
6,U8355,N8419 N15771 N1431 N5888 N18663 N24123 N22130 ...,N51346-0 N33848-0 N15132-0 N10688-0 N6342-0 N6...
7,U46596,N47438 N20950 N21317 N5469,N7821-0 N24898-0 N12029-0 N13579-0 N42977-0 N3...
8,U79199,N37083 N459 N29499 N38118 N37378 N24691 N27235...,N51048-1 N64094-0 N13907-0 N39010-0
9,U53231,N58936 N15919 N11917 N2153 N55312 N13008 N4142...,N53585-1 N55689-0


In [ ]:
#join the users in different experssions to form user with multiple impressions
users_behavior = users_behavior.groupby(['User ID', 'History'], as_index = False).agg({'Impressions': ' '.join})

In [ ]:
users_behavior

,User ID,History,Impressions
0,U100,N20121 N33998 N45954 N55743 N50095 N18870 N534...,N61235-0 N54489-0 N42597-0 N7800-1 N61408-0 N2...
1,U1000,N29641 N1789 N41244,N16148-0 N43628-0 N13094-0 N32774-0 N57034-0 N...
2,U10001,N47937 N51706 N56447 N61319 N27644 N18030 N272...,N42860-0 N62750-0 N33025-0 N3167-0 N51975-0 N4...
3,U10003,N39074 N26619 N31431 N1282 N61052 N41668 N5083...,N58710-0 N39770-0 N33964-0 N64009-0 N36692-0 N...
4,U10008,N23912 N30148 N46754 N33117 N9619 N36526 N4449...,N24272-0 N8016-0 N43595-0 N732-0 N8929-0 N3567...
...,...,...,...
49103,U9993,N47458 N14114,N28918-0 N53515-0 N34869-0 N37204-0 N15574-0 N...
49104,U9995,N61471 N61704 N49103 N63554 N44399 N32117 N218...,N47098-0 N50872-0 N44698-0 N55943-0 N496-0 N15...
49105,U9996,N28296 N4719 N60340 N31165 N8448,N287-1 N64094-0 N51048-0 N39010-0 N38779-0 N15...
49106,U9997,N46759 N46990 N9072 N11727 N50744 N38367 N3333...,N35273-0 N4510-0 N48410-1 N29170-0 N26262-0 N5...


In [ ]:
#make a user history to be made of impressions and history
users_behavior['History'] = users_behavior['History'] + ' ' + users_behavior['Impressions']
#drop the impressions
users_behavior.drop('Impressions', inplace=True, axis=1)

In [ ]:
#let the user id be the indexes
users_behavior.reset_index


<bound method DataFrame.reset_index of       User ID                                            History
0        U100  N20121 N33998 N45954 N55743 N50095 N18870 N534...
1       U1000  N29641 N1789 N41244 N16148-0 N43628-0 N13094-0...
2      U10001  N47937 N51706 N56447 N61319 N27644 N18030 N272...
3      U10003  N39074 N26619 N31431 N1282 N61052 N41668 N5083...
4      U10008  N23912 N30148 N46754 N33117 N9619 N36526 N4449...
...       ...                                                ...
49103   U9993  N47458 N14114 N28918-0 N53515-0 N34869-0 N3720...
49104   U9995  N61471 N61704 N49103 N63554 N44399 N32117 N218...
49105   U9996  N28296 N4719 N60340 N31165 N8448 N287-1 N64094...
49106   U9997  N46759 N46990 N9072 N11727 N50744 N38367 N3333...
49107   U9999  N64503 N20039 N447 N10865 N59026 N12972 N25577...

[49108 rows x 2 columns]>

In [ ]:
users_behavior.head()

,User ID,History
0,U100,N20121 N33998 N45954 N55743 N50095 N18870 N534...
1,U1000,N29641 N1789 N41244 N16148-0 N43628-0 N13094-0...
2,U10001,N47937 N51706 N56447 N61319 N27644 N18030 N272...
3,U10003,N39074 N26619 N31431 N1282 N61052 N41668 N5083...
4,U10008,N23912 N30148 N46754 N33117 N9619 N36526 N4449...


In [ ]:
dict_user = users_behavior

In [ ]:
dict_user = dict_user.set_index('User ID').T.to_dict('list')


In [ ]:
# dict_user = dict_user.set_index('User ID').T.to_dict('list')
# user_history = pd.DataFrame(columns = ["user_id", "history", "verdict"])
# read_news = pd.DataFrame(columns = ["user_id", "history"])
# read = ""
# import time
# i=0
# for row in users_behavior.itertuples():
#   read = ""
#   hist = row.History
#   user_id = row.User_ID
#   hist = list(hist.split(" "))
#   if(len(hist)<50):
#     continue
#   i+=1
#   program_starts = time.time()
#   for h in hist:
#     if (h[-2]=="-"):
#       h = h[:-2]
#       if (h[-1]=="1"):
#         user_history.loc[len(user_history.index)] = [user_id, h, "1"]
#         read = read + " " + h
#       else:
#         user_history.loc[len(user_history.index)] = [user_id, h, "-1"]
#     else:
#       user_history.loc[len(user_history.index)] = [user_id, h, "1"]
#       read = read + " " + h 
#   read_news.loc[len(read_news.index)] = [user_id, read]


In [ ]:
user_history = pd.DataFrame(columns = ["user_id", "history", "verdict"])
read = ""
import time
i=0
read = ""
read_news = {}
program_starts = time.time()   
for row in dict_user:
    read = ""
    user_id = row
    hist = dict_user[row]
    history = ("".join(hist)).split(" ")
    if(len(history)<50):
      continue
    for h in history:
      if (h[-2]=="-"):
         h = h[:-2]
         if (h[-1]=="1"):
            user_history.loc[len(user_history.index)] = [user_id, h, "1"]
         else:
            user_history.loc[len(user_history.index)] = [user_id, h, "-1"]
         read = read + " " + h
      else:
        user_history.loc[len(user_history.index)] = [user_id, h, "1"]
        read = read + " " + h
    read_news[i] = { "user_id": user_id, "history": read}
    i = i + 1
read_news = pd.DataFrame.from_dict(read_news, "index")


In [ ]:
read_news = pd.DataFrame.from_dict(read_news, "index")

In [ ]:
read_news

,user_id,history
0,U100,N20121 N33998 N45954 N55743 N50095 N18870 N53...
1,U1000,N29641 N1789 N41244 N16148 N43628 N13094 N327...
2,U10001,N47937 N51706 N56447 N61319 N27644 N18030 N27...
3,U10003,N39074 N26619 N31431 N1282 N61052 N41668 N508...
4,U10008,N23912 N30148 N46754 N33117 N9619 N36526 N444...
...,...,...
2937,U17549,N20263 N40692 N64358 N13429 N33560 N45765 N26...
2938,U17553,N10818 N48582 N44361 N55326 N17432 N44598 N20...
2939,U17558,N1150 N31177 N27167 N51706 N21628 N14102 N392...
2940,U17560,N56253 N29177 N16290 N61319 N5978 N51818 N266...


In [ ]:
user_history

,user_id,history,verdict
0,U100,N20121,1
1,U100,N33998,1
2,U100,N45954,1
3,U100,N55743,1
4,U100,N50095,1
...,...,...,...
553784,U17563,N35773,-1
553785,U17563,N16096,-1
553786,U17563,N31893,-1
553787,U17563,N25160,-1


In [ ]:
read_news

In [ ]:
#remove those news may be read twice
print(len(user_history))
user_history = user_history.drop_duplicates(subset=["user_id", "history"], keep=False)
print(len(user_history))

553789
424757


In [ ]:
pivoted_user = user_history.pivot("history", "user_id", "verdict").fillna(0)
pivoted_user_dummy = pivoted_user.copy().fillna(0)

In [ ]:
cosine_sim = cosine_similarity(pivoted_user_dummy, pivoted_user_dummy)
cosine_sim = pd.DataFrame(cosine_sim, index=pivoted_user.index, columns=pivoted_user.index)

In [ ]:
def similar_news(news_name):
    similar_score = (cosine_sim[news_name]).sort_values(ascending=False).to_frame().reset_index()
    similar_score.drop(similar_score.loc[similar_score['history'] == news_name].index, inplace=True)
    similar_score.columns = ["News ID", "distance"]
    return similar_score

In [ ]:
def remove_read_news(similar_score, history):
  recommended_news_id = similar_score["News ID"].values.tolist()
  for news in recommended_news_id:
    if news in history:
      similar_score.drop(similar_score.loc[similar_score['News ID'] == news].index, inplace=True)
  return similar_score[0:5]

In [ ]:
news_data = data_before

In [ ]:
def get_news(news, news_data):
    unavailable = []
    news_list =  []
    _newsinfo = pd.DataFrame(columns = ["News ID", "Category", "Title", "Abstract"])
    data_news_id = news_data["News ID"].values.tolist()
    for news_name in news:
      if news_name in data_news_id:
        data = news_data.loc[news_data['News ID'] == news_name]
        _newsinfo = _newsinfo.append(data, ignore_index = True)
      else:
        unavailable.append(news_name)
    news_list.append(unavailable)
    news_list.append(_newsinfo)
    return news_list

In [ ]:
_news = read_news.set_index("user_id")

In [ ]:
_news

In [ ]:
user_id = "U100"
read_news
picked_user = _news.loc[user_id]["history"]
history = picked_user.split(" ")

In [ ]:
def getRandomNews(news_ids):
  user_read = []
  if "" in news_ids:
     news_ids.remove("")
  for i in range(10):
    news = random.choice(news_ids)
    user_read.append(news)
    news_ids.remove(news)
  return user_read

In [ ]:
user_news_history = getRandomNews(history)
user_news_history

In [ ]:
def collaborative_method(user_news_history, history):
    recommended = pd.DataFrame(columns = ["News ID", "distance"])
    i=0
    the_ids =[]
    for news in user_news_history:
        newly_recommended = similar_news(news)
        if(i == 0):
          top_5 = remove_read_news(newly_recommended, history)
          recommended = recommended.append(top_5, ignore_index = True).sort_values(by=['distance'], ascending=False)
          recommended_ids = top_5["News ID"].values.tolist()
          history = history + recommended_ids
        else:
          top_5 = remove_read_news(newly_recommended, history)
          recommended_ids = top_5["News ID"].values.tolist()
          history = history + recommended_ids
          recommended = recommended.append(top_5, ignore_index = True).sort_values(by=['distance'], ascending=False)
        i+=1
    return recommended
collaborative_recommended = collaborative_method(user_news_history, history)
collaborative_recommended

,News ID,distance
0,N4912,0.458277
1,N61214,0.451239
2,N53318,0.449034
3,N35729,0.443563
4,N3930,0.442886
5,N50786,0.353553
6,N32955,0.353553
7,N53465,0.353553
8,N813,0.353553
9,N8122,0.353553


### **The content based recommender**

In [ ]:
userComments = userComments.query("text.str.len() >=200 ", engine="python")
userComments.dropna(inplace=True)

In [ ]:
len(userComments["text"])

16568

In [ ]:
corpus = []

for words in news_data["Abstract"]:
    corpus.append(words.split())

for words in userComments["text"]:
    corpus.append(words.split())

In [ ]:
def delete_random_items(input_list, n):
    to_delete = set(random.sample(range(len(input_list)), n))
    return [x for i,x in enumerate(input_list) if not i in to_delete]
len(corpus)


In [ ]:
# Randomly delete 2000 elements from the list
corpus = delete_random_items(corpus, 23800) 

In [ ]:
len(corpus)

In [ ]:
main_path = "/content/googlenewsvectorsnegative300.zip" # Specify the path

# Check whether the specified path exists or not
if os.path.exists(main_path)== False:
  ! kaggle datasets download leadbest/googlenewsvectorsnegative300
  ! unzip /content/googlenewsvectorsnegative300.zip


In [ ]:
# Load Word2Vec model (trained on an enormous Google corpus)
google_word2vec = KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin', binary = True) 
# Check dimension of word vectors
print(google_word2vec)

In [ ]:
pip install --upgrade gensim

In [ ]:
# Training our corpus with Google Pretrained Model
#create word to vector model 
model = Word2Vec(vector_size = 300, min_count = 1)
model.build_vocab(corpus)
model.wv.vectors_lockf = np.ones(len(model.wv))

model.wv.intersect_word2vec_format('/content/GoogleNews-vectors-negative300.bin', binary=True, lockf=1.0)
model.train(corpus, total_examples=model.corpus_count, epochs = 5)

(9297488, 10434060)

In [ ]:
#Building TFIDF model and calculate TFIDF score

tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df = 5)
tfidf.fit(news_data['Abstract'])

# Getting the words from the TF-IDF model

tfidf_list = dict(zip(tfidf.get_feature_names(), list(tfidf.idf_)))
tfidf_feature = tfidf.get_feature_names() # tfidf words/col-names

In [ ]:
# @title Another Content based_approach
# vocabulary = model.wv.key_to_index
# w2v_headline = []
# for i in news_data["Abstract"]:
#     w2Vec_word = np.zeros(300, dtype="float32")
#     for word in i.split():
#         if word in vocabulary:
#           w2Vec_word = np.add(w2Vec_word, model.wv[word])
#     w2Vec_word = np.divide(w2Vec_word, len(i.split()))
#     w2v_headline.append(w2Vec_word)

# w2v_headline = np.array(w2v_headline)
# w2v_headline =np.nan_to_num(w2v_headline)


In [ ]:
#@title Recommending
def avg_w2v_based_model(news_id, num_similar_items):
    abstract = (news_data.loc[news_data['News ID'] ==news_id ])
    row_index =  abstract.index[0]
    couple_dist = pairwise_distances(w2v_headline, w2v_headline[row_index].reshape(1,-1))
    indices = np.argsort(couple_dist.ravel())[0:num_similar_items]
    df = pd.DataFrame({"News ID": news_data['News ID'][indices].values, "distance": couple_dist[indices].ravel()})
    return df.iloc[1:,]

news_idd =  "N2146"
avg_w2v_based_model(news_idd, 10)


Building the content based recommender.

In [ ]:
# @title Default title text
# Building TF-IDF Word2Vec

# Storing the TFIDF Word2Vec embeddings
tfidf_vectors = []; 
line = 0;
# for each article abstract
for desc in corpus:
  
    # Word vectors are of zero length (Used 300 dimensions)
    sent_vec = np.zeros(300)

    # num of words with a valid vector in the book descriptionAbstract
    weight_sum = 0

    # for each word in the article abstract
    for word in desc: 
        if word in model.wv.key_to_index and word in tfidf_feature:
            vec = model.wv[word]
            tf_idf = tfidf_list[word] * (desc.count(word) / len(desc))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_vectors.append(sent_vec)
    line += 1

In [ ]:
# def delete_random_items(input_list, n):
#     to_delete = set(random.sample(range(len(input_list)), n))
#     return [x for i,x in enumerate(input_list) if not i in to_delete]
# tfidf_vectors = delete_random_items(tfidf_vectors, 30000) 


In [ ]:
users_behavior.column =["user_id", "history" ]
users_behavior = users_behavior.reset_index()

In [ ]:
# cosine_similarities = cosine_similarity(tfidf_vectors, tfidf_vectors)

In [ ]:
news_data

In [ ]:
news_data.index = range(len(news_data))

In [ ]:
news_data

In [ ]:
news_data = data_before

In [ ]:
def recommendations(news_id, news_data, limit ):
    # finding cosine similarity for the vectors
    abstract = (news_data.loc[news_data['News ID'] == news_id])
    idx = abstract.index[0]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sim_scores[:limit]
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:10]
    news_indices = [i[0] for i in sim_scores]
    distance = [i[1] for i in sim_scores]
    df = pd.DataFrame({"News ID": news_data['News ID'][news_indices].values, "distance": distance})
    return df.iloc[0:,]

In [ ]:
def content_based_method(user_news_history, available_news, limit, history):
    recommended = pd.DataFrame(columns = ["News ID", "distance"])
    i=0
    for news in user_news_history:
        newly_recommended = recommendations(news, available_news, limit)
        if(i == 0):
          top_5 = remove_read_news(newly_recommended, history)
          recommended = recommended.append(top_5, ignore_index = True).sort_values(by=['distance'], ascending=False)
          recommended_ids = top_5["News ID"].values.tolist()
          history = history + recommended_ids
        else:
          top_5 = remove_read_news(newly_recommended, history)
          recommended_ids = top_5["News ID"].values.tolist()
          history = history + recommended_ids
          recommended = recommended.append(top_5, ignore_index = True).sort_values(by=['distance'], ascending=False)
        i+=1
    return recommended

In [ ]:
user_id = "U100"
seen_news = user_impressions.loc[user_impressions['User ID'] == user_id]
news_ids = seen_news["History"].values.tolist()[0]
history = ("".join(news_ids)).split(" ")
user_news_history = getRandomNews(history)
user_news_history


In [ ]:
limit = 40000
content_recommended = content_based_method(user_news_history, news_data, limit, history)
collaborative_recommended = collaborative_method(user_news_history, history)
print(content_recommended)
print(collaborative_recommended)

In [ ]:
content_id = content_recommended["News ID"].values.tolist()
collab_id = collaborative_recommended["News ID"].values.tolist()

In [ ]:
print("="*30, "THESE ARE THE TOP 15 CONTENT BASED RECOMMENDED NEWS", "="*30)
content_based = get_news(content_id, news_data)[1]
content_based.head(15)

============================== THESE ARE THE TOP 15 CONTENT BASED RECOMMENDED NEWS ==============================


,News ID,Category,Title,Abstract
0,N3458,health,What to Do If You Think You Have a Tick-Borne ...,answer key question may process getting diagno...
1,N50852,video,WSJ uncovers even more pictures of Trump & ind...,lead panel discus
2,N39951,finance,How the National Debt Hits Your Wallet Every Day,might want adjust spending investment
3,N3471,finance,I looked into 'house hacking' to live for free...,house hacking utilize primary residence help e...
4,N37585,news,Russia has retaliation ready if U.S. quits Ope...,russia retaliation ready quits open sky treaty...
5,N23153,movies,Jennifer Aniston says Marvel's green-screen mo...,comment come day martin scorsese compared mcu ...
6,N47760,autos,Daetwan Robinson charged in fatal hit-and-run ...,milwaukee daetwan robinson charged connection ...
7,N41018,sports,Report: Dodgers expected to pursue trade for I...,yet known whether indian would willing trade b...
8,N36563,sports,Blackhawks still waiting for Jonathan Toews an...,chicago question hung air second blackhawks co...
9,N28752,news,Passenger dies in crash caused by wrong-way dr...,passenger car going wrong way kansa city north...


In [ ]:
print("="*30, "THESE ARE THE TOP 15 COLLAB-BASED RECOMMENDED NEWS", "="*30)
collab_based = get_news(collab_id, news_data)[1]
collab_based.head(15)

============================== THESE ARE THE TOP 15 COLLAB-BASED RECOMMENDED NEWS ==============================


,News ID,Category,Title,Abstract
0,N4912,movies,New Movies and TV Shows You'll Be Able to Cozy...,movie watcher divide two camp november roll ar...
1,N61214,news,SpaceX set to launch another 60 Starlink inter...,initial service expected begin next year globa...
2,N53318,foodanddrink,The best independent bakery in every state,want best bread flakiest pastry comforting cre...
3,N35729,news,Porsche launches into second story of New Jers...,porsche went airborne median tom river causing...
4,N3930,sports,Insiders predict: NFL week 10 winners,entered portion season peeking playoff picture...
5,N8122,sports,AP Top 25: Ohio State jumps Clemson to 3rd; Wi...,ohio state edged past clemson associated press...
6,N46148,finance,China is breeding giant pigs that are as heavy...,farm deep southern region china life big pig h...
7,N32955,news,White Shelton middle school student ejected fr...,white shelton middle school student kicked nat...
8,N813,news,Washington voters narrowly rejecting affirmati...,controversial measure would reverse decade lon...
9,N50786,news,Trump in Kentucky: Protesters and Trump suppor...,many house democrat impeachment inquiry latest...


In [ ]:
hybrid_recommender = pd.concat([collaborative_recommended, content_recommended ])

In [ ]:
hybrid_recommender = hybrid_recommender.groupby(['News ID'], as_index=False )['distance'].agg('sum').sort_values(by=['distance'],  ascending=False)

,News ID,distance
40,N3458,0.977018
73,N50852,0.976184
53,N39951,0.974187
41,N3471,0.969349
47,N37585,0.967233
...,...,...
62,N46622,0.167836
9,N15797,0.167836
63,N47631,0.147442
1,N10414,0.138458


In [ ]:
print("="*30, "THESE ARE THE TOP 15 CONTENT BASED RECOMMENDED NEWS", "="*30)
hybrid_ID = hybrid_recommender["News ID"].values.tolist()
hybrid_recommender_based = get_news(content_id, news_data)[1]
content_based.head(15)

Evaluating the recommender System

In [ ]:
def users_info(user_id):
  list_data = []
  user_id = "U100"
  seen_news = user_history.loc[user_history['user_id'] == user_id].drop_duplicates()
  history = seen_news["history"].values.tolist()
  user_news = get_news(history, news_data)
  available_news = user_news[1]
  removed_news = user_news[0]
  seen_news = seen_news.drop(seen_news[seen_news["history"].isin(removed_news)].index.tolist()).reset_index(drop=True)
  history = seen_news["history"].values.tolist()
  seen_news["verdict"] = seen_news['verdict'].replace({'-1': '0'})
  info = [seen_news, available_news]
  return info

In [ ]:
# #get user all the news he has rated as our database of news drop those not in our main database

# user_id = "U100"
# seen_news = user_history.loc[user_history['user_id'] == user_id].drop_duplicates()
# history = seen_news["history"].values.tolist()
# user_news = get_news(history, news_data)
# available_news = user_news[1]
# removed_news = user_news[0]
# seen_news = seen_news.drop(seen_news[seen_news["history"].isin(removed_news)].index.tolist()).reset_index(drop=True)
# history = seen_news["history"].values.tolist()
# seen_news["verdict"] = seen_news['verdict'].replace({'-1': '0'})
# history
# # y_true = seen_news['verdict'].to_frame()
# # y_true = y_true[y_true.columns[0]].values.tolist()
# # y_true = list(map(int, y_true))


In [ ]:
#group by clicks
def click_and_noneclick(user):
    seen_news = users_info(user)
    clicked  = seen_news[0].loc[seen_news[0]['verdict'] == "1"].reset_index(drop=True)
    not_clicked = seen_news[0].loc[seen_news[0]['verdict'] == "0"].reset_index(drop=True)
    clicked = clicked.truncate(before=0, after=25)
    not_clicked = not_clicked.truncate(before=0, after=25)
    list_click = [clicked, not_clicked, seen_news]
    return list_click

In [ ]:
#trancate to make a small dataset test_list = list(map(int, y_train))
def y_train_and_x_train(user):
    clicks = click_and_noneclick(user)
    join = [clicks[0], clicks[1]]
    user_news_data = pd.concat(join)
    x_train = user_news_data["history"].values.tolist()
    y_train = user_news_data['verdict'].to_frame()
    y_train = y_train[y_train.columns[0]].values.tolist()
    y_train = [int(x) for x in y_train]
    data = [y_train, x_train, clicks[2]]
    return data

 **CONTENT-BASED RECOMMENDER EVALUATION**

In [ ]:
def get_predictions(test_news, available_news, available_news_data, y_true):
    empty = [""]
    evaluation_content_recommended = content_based_method(test_news, available_news_data, len(available_news_data), empty)
    ids = evaluation_content_recommended["News ID"].values.tolist()
    recommended_list = evaluation_content_recommended["News ID"].values.tolist()
    y_predicted = []
    for news in recommended_list:
      koo = available_news.loc[available_news['history'] == news]
      y_predicted.append(int(koo["verdict"].values[0]))
    y_pred = y_predicted[:len(test_news)]
    evaluation = [y_pred, y_true]
    return evaluation

In [ ]:
def get_y_pred(user):
  data = y_train_and_x_train(user)
  y_train = data[0]
  test_news = data[1]
  available_news = data[2][0]
  available_news_data = data[2][1]
  return get_predictions(test_news, available_news, available_news_data, y_train)
evaluation = get_y_pred("U100")

In [ ]:
evaluation[0]

[0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0]

In [ ]:
import statistics as st
user_evaluation = read_news["user_id"].values.tolist()[:100]
ndcg_k_5 = []
ndcg_k_10 = []
map = []
for user in user_evaluation:
  evaluation = get_y_pred(user)
  y_pred = [evaluation[0]]
  y_true = [evaluation[1]]
  ndcg_k_5.append(ndcg_score(y_pred, y_true, k =5))
  ndcg_k_10.append(ndcg_score(y_pred, y_true, k =10))
  y_true = np.squeeze(y_true,axis = 0)
  y_pred = np.squeeze(y_pred,axis = 0)
  map.append(average_precision_score(y_pred, y_true))
print("the average ndcg for users k = 5", st.mean(ndcg_k_5))
print("the average ndcg for users k = 10", st.mean(ndcg_k_10))
print("the average map for users", st.mean(map))


In [ ]:
user = user_evaluation[12]
name = get_y_pred(user)

In [ ]:
print(name[1])
print(name[0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]


the ndcg 0.11111111111111108
the ndcg 0.17122161308654427
the ndcg 0.11262626262626263


In [ ]:
empty = [""]
eval_collaborative_recommended = collaborative_method(test_news, empty)
eval_collaborative_recommended

In [ ]:
recommended_list = eval_collaborative_recommended["News ID"].values.tolist()
y_predicted = []
news_names = []
for news in recommended_list:
  news = seen_news.loc[seen_news['history'] == news]
  if(len(news)>0):  
    y_predicted.append(int(news["verdict"].values[0]))
y_p = y_predicted[:len(test_news)]

,user_id,history,verdict
0,U100,N20121,1
1,U100,N33998,1
2,U100,N45954,1
3,U100,N55743,1
4,U100,N50095,1
...,...,...,...
110,U100,N30475,-1
111,U100,N52752,-1
112,U100,N60904,-1
113,U100,N40839,-1


In [ ]:
y_pred = [y_p]
y_true = [y_t]

In [ ]:
import numpy as np
from sklearn.metrics import ndcg_score, average_precision_score
print("the ndcg", ndcg_score(y_pred, y_true, k =5))
print("the ndcg", ndcg_score(y_pred, y_true, k =10))
y_true = np.squeeze(y_true,axis = 0)
y_pred = np.squeeze(y_pred,axis = 0)
print("the ndcg", average_precision_score(y_pred, y_true))

the ndcg 0.0
the ndcg 0.0
the ndcg 0.09090909090909091


In [ ]:
hybrid_recommender = pd.concat([eval_collaborative_recommended, evaluation_content_recommended ])
hybrid_recommender = hybrid_recommender.groupby(['News ID'], as_index=False )['distance'].agg('sum').sort_values(by=['distance'],  ascending=False)
hybrid_ID = hybrid_recommender["News ID"].values.tolist()
recommended_list = eval_collaborative_recommended["News ID"].values.tolist()
y_predicted = []
for news in hybrid_ID:
  news = seen_news.loc[seen_news['history'] == news]
  if(len(news)>0):  
    y_predicted.append(int(news["verdict"].values[0]))
y_p = y_predicted[:len(test_news)]

In [ ]:
y_pred = [y_p]
y_true = [y_t]
y_pred

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0]]

In [ ]:
import numpy as np
from sklearn.metrics import ndcg_score, average_precision_score
print("the ndcg", ndcg_score(y_pred, y_true, k =5))
print("the ndcg", ndcg_score(y_pred, y_true, k =10))
y_true = np.squeeze(y_true,axis = 0)
y_pred = np.squeeze(y_pred,axis = 0)
print("the ndcg", average_precision_score(y_pred, y_true))

the ndcg 0.0
the ndcg 0.0
the ndcg 0.22727272727272727
